In [11]:
# Imports
import torch
import torch.nn as nn
from models import SiameseNetwork_Images, SiameseNetwork_Full, PairedMedicalDataset_Images, PairedMedicalDataset_Full
import shap
import glob
import os
import pandas as pd
import numpy as np
from monai.transforms import (
    Resize,
    ScaleIntensity,
    Compose
)
import nibabel as nib

ImportError: cannot import name 'SiameseNetwork_Images' from 'models' (c:\Users\P095550\OneDrive - Amsterdam UMC\Documenten\GitHub\CRLM-morph-features\models.py)

# Model evaluation

## Images only model

In [ ]:
# Instantiate the base model (e.g., ResNet or other feature extractor)

resnet_model = torch.hub.load('Warvito/MedicalNet-models', 'medicalnet_resnet10')

# Remove the final classification layer (fc) to keep only the encoder part
encoder = nn.Sequential(*list(resnet_model.children())[:-1])

model = SiameseNetwork(encoder)
model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))
model.eval()

## Full model

In [ ]:
# Instantiate the base model (e.g., ResNet or other feature extractor)

resnet_model = torch.hub.load('Warvito/MedicalNet-models', 'medicalnet_resnet10')

# Remove the final classification layer (fc) to keep only the encoder part
encoder = nn.Sequential(*list(resnet_model.children())[:-1])

model = SiameseNetwork(encoder)
model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))
model.eval()

### Make predictions


In [53]:
class SiameseWrapper(nn.Module):
    def __init__(self, model):
        super(SiameseWrapper, self).__init__()
        self.model = model

    def forward(self, input):
        image1, image2, metadata = input
        return self.model(image1, image2, metadata)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_dir = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_scans"
clinical_data_dir = "C:/Users/P095550/OneDrive - Amsterdam UMC/Documenten/GitHub/CRLM-morph-features"
nifti_images = sorted(glob.glob(os.path.join(data_dir, "*.nii.gz")))   

# Create pairs (e.g., first and second file are paired)
image_pairs = [(nifti_images[i], nifti_images[i + 1]) for i in range(0, len(nifti_images) - 1, 2)]

pd_metadata = pd.read_csv(os.path.join(clinical_data_dir, "training_input.csv"))
all_metadata = torch.tensor(pd_metadata.values.tolist())

pd_labels = pd.read_csv(os.path.join(clinical_data_dir, "training_labels.csv"))
all_labels = torch.tensor(pd_labels.values.tolist()) #Fill in correct path. response, PFS, and OS

Using device: cpu


In [5]:
dataset = PairedMedicalDataset(
    image_pairs, all_metadata, all_labels, transform=[ScaleIntensity(), Resize((64, 256, 256), mode="trilinear")]
)

# Create DataLoaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)


In [9]:
for i, (image1, image2, metadata, label) in enumerate(data_loader):
    with torch.no_grad():
        # Forward pass through the model
        output = model(image1, image2, metadata)
        probs = torch.sigmoid(output)
    print(probs)

metatensor([[0.4648, 0.5368, 0.4234, 0.7841, 0.5976, 0.5243, 0.7695, 0.5624, 0.5321]])
metatensor([[0.5212, 0.5548, 0.4162, 0.7993, 0.6100, 0.4530, 0.7992, 0.5509, 0.4867]])
metatensor([[0.4251, 0.2870, 0.2183, 0.9963, 0.9055, 0.6109, 0.9938, 0.8718, 0.9511]])
metatensor([[0.4378, 0.4328, 0.5289, 0.8097, 0.6394, 0.6598, 0.8072, 0.6486, 0.6172]])
metatensor([[0.4190, 0.2747, 0.2222, 0.9965, 0.9095, 0.6159, 0.9941, 0.8803, 0.9549]])
metatensor([[0.4574, 0.5144, 0.4293, 0.7928, 0.5997, 0.5310, 0.7726, 0.5715, 0.5349]])
metatensor([[0.4685, 0.5515, 0.4179, 0.7783, 0.6079, 0.5294, 0.7706, 0.5585, 0.5298]])
metatensor([[0.4930, 0.5665, 0.3967, 0.7840, 0.5926, 0.4583, 0.7767, 0.5344, 0.4833]])
metatensor([[0.5485, 0.5451, 0.4688, 0.8128, 0.6254, 0.4677, 0.8322, 0.5704, 0.4864]])
metatensor([[0.4905, 0.5431, 0.2512, 0.9462, 0.7551, 0.5013, 0.9299, 0.6360, 0.7539]])
metatensor([[0.4671, 0.5513, 0.3740, 0.7870, 0.5925, 0.4820, 0.7645, 0.5478, 0.5285]])
metatensor([[0.5551, 0.5530, 0.4977, 0.8184

KeyboardInterrupt: 

In [ ]:
#data

# Create an iterator for the DataLoader
data_iter = iter(data_loader)

# Extract one batch
batch = next(data_iter)

# Unpack the batch (if applicable)
img1, img2, metadata, labels = batch


# Dummy image pair input
image1_test = img1[0].to("cuda")
image2_test = img2[0].to("cuda")
metadata_test = metadata[0].to("cuda")


# Background for SHAP (used by DeepExplainer)
image1_bg = img1[1:].to("cuda")
image2_bg = img2[1:].to("cuda")
metadata_bg = metadata[1:].to("cuda")


In [41]:
print(type(image2_test))

<class 'monai.data.meta_tensor.MetaTensor'>


In [ ]:
wrapped_model = SiameseWrapper(model).to("cuda")

# Use DeepExplainer for PyTorch models
explainer = shap.GradientExplainer(
    wrapped_model,
    data=(image1_bg, image2_bg, metadata_bg)
)

shap_values = explainer.shap_values((image1_test, image2_test, metadata_test))



KeyboardInterrupt



### Integrated gradients

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from captum.attr import IntegratedGradients
from torchvision import models


# Instantiate the base model (e.g., ResNet or other feature extractor)

resnet_model = torch.hub.load('Warvito/MedicalNet-models', 'medicalnet_resnet10')

# Remove the final classification layer (fc) to keep only the encoder part
encoder = nn.Sequential(*list(resnet_model.children())[:-1])

model = SiameseNetwork(encoder)
model.load_state_dict(torch.load("best_model.pth", map_location='cpu'))

wrapped_model = SiameseWrapper(model)

wrapped_model.eval()


# Sample input
image_input = img1[0]
clinical_input = metadata[0]

# Use Integrated Gradients to explain the image input
ig = IntegratedGradients(wrapped_model)

# Explain the image input (we need to make sure that the image tensor requires gradients)
image_input.requires_grad = True
image_attr, _ = ig.attribute((img1[0], img2[0], metadata[0]), target=0, return_convergence_delta=True)

# Now explain the clinical input (again, ensure the tensor requires gradients)
clinical_input.requires_grad = True
clinical_attr, _ = ig.attribute(clinical_input, target=0, return_convergence_delta=True)

# Output the explanations
print("Image Attribution:", image_attr)
print("Clinical Data Attribution:", clinical_attr)

# You can visualize or process the attributions further


Using cache found in C:\Users\P095550/.cache\torch\hub\Warvito_MedicalNet-models_main


TypeError: SiameseWrapper.forward() takes 2 positional arguments but 4 were given